In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI AUTOMATIQUE

import os
import subprocess

COMFY_PATH = "/workspace/ComfyUI"
MANAGER_PATH = os.path.join(COMFY_PATH, "custom_nodes/ComfyUI-Manager")

print("🚀 Vérification de l'installation de ComfyUI...")

# 1. GESTION DU CODE SOURCE (GIT)
if os.path.exists(COMFY_PATH):
    print("📂 ComfyUI détecté. Mise à jour en cours...")
    try:
        # git fetch récupère les infos, git reset --hard force l'état local
        subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True, check=True)
        print("✅ Mise à jour Code ComfyUI réussie.")
    except Exception as e:
        print(f"⚠️ Erreur lors de la mise à jour : {e}")
        print("   -> Tentative de git pull classique...")
        subprocess.run(f"cd {COMFY_PATH} && git pull", shell=True)
else:
    print("📥 Première installation (Clonage)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)

# 2. INSTALLATION / UPDATE DU MANAGER (Recommandé)
if not os.path.exists(MANAGER_PATH):
    print("📥 Clonage de ComfyUI-Manager...")
    subprocess.run(f"cd {COMFY_PATH}/custom_nodes && git clone https://github.com/ltdrdata/ComfyUI-Manager.git", shell=True)
else:
    print("📂 Mise à jour ComfyUI-Manager...")
    subprocess.run(f"cd {MANAGER_PATH} && git pull", shell=True)

# 3. INSTALLATION DES DÉPENDANCES (PIP)
print("📦 Installation / Mise à jour des dépendances Python...")

# Installation des dépendances principales de ComfyUI
subprocess.run(f"cd {COMFY_PATH} && pip install -r requirements.txt", shell=True)

# Installation des dépendances du Manager (si le fichier existe)
if os.path.exists(f"{MANAGER_PATH}/requirements.txt"):
    print("📦 Installation dépendances Manager...")
    subprocess.run(f"cd {MANAGER_PATH} && pip install -r requirements.txt", shell=True)

# Installation de xformers (Souvent nécessaire pour la vitesse sur Vast/RunPod)
print("⚡ Vérification de xformers...")
subprocess.run("pip install xformers", shell=True)

print("✅ Installation terminée. Prêt à lancer.")


INSTALLATION CUSTOM NODES

In [ ]:
# === INSTALLATION CUSTOM NODES (Fix NumPy + Nodes + Modèles + Patch) ===

import os
import subprocess
import sys

COMFY_DIR = "/workspace/ComfyUI"
CUSTOM_NODES_DIR = f"{COMFY_DIR}/custom_nodes"

print("🚀 DÉMARRAGE DE L'INSTALLATION FLUX & TOOLS...")

# --- 0. FIX CRITIQUE NUMPY & DÉPENDANCES SYSTÈME ---
print("🛠️ Préparation du système (Fix NumPy < 2.0)...")

# 1. Installer Aria2c
try:
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "aria2"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except:
    pass

# 2. FORCER LE DOWNGRADE DE NUMPY (Crucial pour éviter le crash InsightFace)
# On désinstalle et réinstalle une version compatible 1.x
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run([sys.executable, "-m", "pip", "install", "numpy<2"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("   ✅ NumPy corrigé (v1.x).")

# --- 1. LISTE DES NODES ---
nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",
    "https://github.com/rgthree/rgthree-comfy",
    "https://github.com/lldacing/ComfyUI_PuLID_Flux_ll",
    "https://github.com/cubiq/ComfyUI_essentials",
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",
    "https://github.com/Gourieff/ComfyUI-ReActor",
    "https://github.com/jags111/efficiency-nodes-comfyui",
    "https://github.com/city96/ComfyUI-GGUF",
    "https://github.com/ltdrdata/was-node-suite-comfyui",
    "https://github.com/1038lab/ComfyUI-QwenVL",
    "https://github.com/rzgarespo/ComfyUI-diffusiondb",
    "https://github.com/BizaNator/ComfyUI_PromptIterator",
    "https://github.com/ClownsharkBatwing/RES4LYF",
    "https://github.com/lquesada/ComfyUI-Inpaint-CropAndStitch",
    "https://github.com/ChangeTheConstants/SeedVarianceEnhancer",
    "https://github.com/PozzettiAndrea/ComfyUI-SAM3",
    "https://github.com/martin-rizzo/ComfyUI-ZImagePowerNodes", 
    "https://github.com/peterkickasspeter-civit/ComfyUI-ZImageTurboProgressiveLockedUpscale",
    
]

# --- 2. INSTALLATION INSIGHTFACE & NODES ---
print("📦 Installation de InsightFace & ONNX (peut prendre 1-2 min)...")
# On force la réinstallation pour être sûr qu'ils se lient au bon NumPy
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "insightface", "onnxruntime-gpu", "albumentations"], 
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except Exception as e:
    print(f"⚠️ Warning InsightFace: {e}")

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur clone {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} ok.")

# --- 3. TÉLÉCHARGEMENT MODÈLES ULTRALYTICS (YOLO) ---
print("\n👁️ Vérification des modèles de détection (YOLO)...")
base_yolo = f"{COMFY_DIR}/models/ultralytics"
bbox_path = os.path.join(base_yolo, "bbox")
segm_path = os.path.join(base_yolo, "segm")
os.makedirs(bbox_path, exist_ok=True)
os.makedirs(segm_path, exist_ok=True)

yolo_models = [
    # BBOX
    ("face_yolov8m.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt", bbox_path),
    ("face_yolov8n.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n.pt", bbox_path),
    ("hand_yolov8s.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt", bbox_path),
    ("Eyes.pt", "https://huggingface.co/Tenofas/ComfyUI/resolve/d79945fb5c16e8aef8a1eb3ba1788d72152c6d96/ultralytics/bbox/Eyes.pt", bbox_path),
    # SEGM
    ("person_yolov8m-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt", segm_path),
    ("yolov8n-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/yolov8n-seg.pt", segm_path),
    ("face_yolov8m-seg_60.pt", "https://huggingface.co/bobmin/face_yolov8m-seg_60/resolve/main/face_yolov8m-seg_60.pt", segm_path)
]

for name, url, dest in yolo_models:
    full_path = os.path.join(dest, name)
    if not os.path.exists(full_path):
        print(f"   ⬇️ YOLO Download : {name}")
        # Méthode 1: Aria2c
        try:
            subprocess.run(["aria2c", "-x", "8", "-s", "8", "-k", "1M", "-d", dest, "-o", name, url], 
                           check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except:
            # Méthode 2: Wget (Fallback plus robuste)
            print(f"   ⚠️ Aria2c échoué, bascule sur wget pour {name}...")
            try:
                subprocess.run(["wget", "-q", "--show-progress", "-O", full_path, url], check=True)
            except Exception as e:
                print(f"   ❌ Échec total pour {name}. Vérifiez votre connexion.")

# --- 4. PATCH REACTOR NSFW ---
print("\n🔓 Application du patch ReActor (NSFW Bypass)...")
reactor_path = f"{CUSTOM_NODES_DIR}/ComfyUI-ReActor/scripts/reactor_sfw.py"
bypass_code = """
from PIL import Image
import logging
import os
from scripts.reactor_logger import logger
MODEL_EXISTS = True
def ensure_nsfw_model(nsfwdet_model_path): return True
def nsfw_image(img_data, model_path: str): return False
"""

if os.path.exists(os.path.dirname(reactor_path)):
    try:
        with open(reactor_path, "w") as f: f.write(bypass_code)
        print("   ✅ Patch appliqué.")
    except Exception as e:
        print(f"   ❌ Erreur patch : {e}")

print("\n🎉 INSTALLATION TERMINÉE ! Veuillez relancer ComfyUI.")

In [ ]:
FILES DOWNLOAD

In [ ]:
# Z-IMAGE TURBO & BASE FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Z-Image & Turbo) ---
db_models = {
    "✨ Nouveautés": [
        ("RealDream Z-Image V1 (FP8)", "https://civitai.com/api/download/models/2507717?type=Model&format=SafeTensor&size=pruned&fp=fp8", "/workspace/ComfyUI/models/diffusion_models", "realDream_zImageV1.safetensors", "civitai"),
    ],
    "🧠 Modèles GGUF (Unet)": [
        ("[GGUF] Z-Image Base BF16", "https://huggingface.co/babakarto/z-image-base-gguf/resolve/main/z_image_base_BF16.gguf?download=true", "/workspace/ComfyUI/models/unet", "z_image_base_BF16.gguf", "hf"),
    ],
    "🧬 Z-Image & Turbo Core": [
        # --- FICHIER AJOUTÉ ---
        ("[10.8 GB] Z-Image Base BF16 (Non-Turbo)", "https://huggingface.co/Comfy-Org/z_image/resolve/main/split_files/diffusion_models/z_image_bf16.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "z_image_bf16.safetensors", "hf"),
        # ----------------------
        ("[11.7 GB] Z-Image Turbo BF16", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/diffusion_models/z_image_turbo_bf16.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "z_image_turbo_bf16.safetensors", "hf"),
        ("[12.6 GB] Z-Image Turbo BF16 NSFW V2", "https://huggingface.co/tewea/z_image_turbo_bf16_nsfw/resolve/main/z_image_turbo_bf16_nsfw_v2.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "z_image_turbo_bf16_nsfw_v2.safetensors", "hf"),
        ("[320 MB] VAE (ae.safetensors)", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors?download=true", "/workspace/ComfyUI/models/vae", "ae.safetensors", "hf"),
        ("[7.5 GB] Qwen 3.4B (Text Enc)", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors?download=true", "/workspace/ComfyUI/models/text_encoders", "qwen_3_4b.safetensors", "hf"),
    ],
    "🧪 LoRAs Personnels (ZIT Training)": [
        ("Grainscape Z-Image", "https://huggingface.co/Danrisi/GrainScape_UltraReal_ZImage/resolve/main/grainscape_zimage.safetensors", "/workspace/ComfyUI/models/loras", "grainscape_zimage.safetensors", "hf"),
        ("Tumblr Minimalism 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/tmblr0ne_minimalism_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "tmblr0ne_minimalism_000002250.safetensors", "hf"),
        ("Tumblr Minimalism 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/tmblr0ne_minimalism_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "tmblr0ne_minimalism_000002750.safetensors", "hf"),
        ("Hairfix 1500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000001500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000001500.safetensors", "hf"),
        ("Hairfix 1750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000001750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000001750.safetensors", "hf"),
        ("Hairfix 2000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000002000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000002000.safetensors", "hf"),
        ("Hairfix 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000002250.safetensors", "hf"),
        ("Vy3sna Woman 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002250.safetensors", "hf"),
        ("Vy3sna Woman 2500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002500.safetensors", "hf"),
        ("Vy3sna Woman 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002750.safetensors", "hf"),
        ("Vy3sna Woman 3000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000003000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000003000.safetensors", "hf"),
        ("Vy3sna Woman 3250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000003250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000003250.safetensors", "hf"),
        ("Vyesna ZIT v1 1750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000001750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000001750.safetensors", "hf"),
        ("Vyesna ZIT v1 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002250.safetensors", "hf"),
        ("Vyesna ZIT v1 2500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002500.safetensors", "hf"),
        ("Vyesna ZIT v1 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002750.safetensors", "hf"),
        ("Vyesna ZIT v1 3000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000003000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000003000.safetensors", "hf"),
        ("Vyesna ZIT v1 3250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000003250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000003250.safetensors", "hf"),
    ],
    "🛠️ Upscale Models": [
        ("4x-UltraSharp", "https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "4x-UltraSharp.pth", "hf"),
        ("RealESRGAN x4plus", "https://huggingface.co/lllyasviel/Annotators/resolve/main/RealESRGAN_x4plus.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "RealESRGAN_x4plus.pth", "hf"),
        ("Remacri Original", "https://huggingface.co/jaideepsingh/upscale_models/resolve/main/remacri_original.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "remacri_original.safetensors", "hf"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '250px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Z-Image)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Z-Image...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue

                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                ret = os.system(cmd)
                if ret == 0: print(f"   ✅ OK.")
                else: print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0: print("⚠️ Aucune case cochée.")
        else: print("\n🎉 Terminé.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🧬 INSTALLATION Z-IMAGE TURBO (Et LoRAs Persos)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
INSTALLATION DEPENDANCES EPHEMERES + LANCEMENT COMFYUI

In [ ]:
# 🚀 DÉMARRAGE COMFYUI (V16 FINAL FIX - SMART & ROBUST)
# Correction : Gère correctement l'absence du module 'soundfile' au premier démarrage.

import os
import subprocess
import sys
import importlib.util

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
os.environ["PIP_ROOT_USER_ACTION"] = "ignore" # Silence warnings

def is_installed(package_name):
    try:
        if importlib.util.find_spec(package_name) is not None:
            return True
    except ImportError:
        pass
    return False

print("🧠 DÉMARRAGE V16 (Robustesse Maximale)...")

# 0. MISE À JOUR PIP (Rapide)
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 1. OUTILS SYSTÈME (Check Audio & Modules)
print("🔧 Vérification outils système...")
try:
    import soundfile
except (OSError, ImportError): # <--- FIX ICI : On attrape les deux erreurs
    print("   -> Installation système (libsndfile, cmake, ffmpeg)...")
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "ffmpeg", "git", "cmake", "aria2", "libsndfile1", "libsndfile1-dev", "build-essential", "ninja-build"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. PYTORCH (Check version)
print("🔥 Vérification PyTorch...")
try:
    import torch
    if "2.5.1" not in torch.__version__:
        raise ImportError
except ImportError:
    print("   -> Installation PyTorch 2.5.1...")
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
        "--index-url", "https://download.pytorch.org/whl/cu124"
    ])

# 3. TRANSFORMERS (Check version pour QwenVL)
print("🤖 Vérification Transformers (<5.0)...")
need_trans_install = True
try:
    import transformers
    # On veut la 4.x, pas la 5.x
    if transformers.__version__.startswith("4."):
        need_trans_install = False
except ImportError:
    pass

if need_trans_install:
    print("   -> Correction Transformers...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers>=4.46.0,<5.0.0", "huggingface_hub>=0.25.0", "accelerate>=0.26.0"])

# 4. PAQUETS STANDARDS (Rapide)
print("📦 Vérification dépendances standards...")
pypi_packages = [
    "protobuf", "bitsandbytes", "ftfy", "google-generativeai", "soundfile", "librosa", 
    "numpy<2", "pydantic-settings", "sentencepiece", "aiohttp", "einops", "av", "torchsde", "scipy",
    "insightface", "onnxruntime-gpu", "kornia", "spandrel", "color-matcher",
    "alembic", "gguf", "segment-anything", "diffusers", "ultralytics", "piexif", "facexlib",
    "comfyui-workflow-templates", "rotary_embedding_torch", "dill", "timm", "simpleeval", "flatten_json",
    "blend_modes", "facenet-pytorch", "mediapipe", "fvcore", "yapf", "omegaconf", "addict", 
    "trimesh[easy]", "pilgram", "rembg", "duckdb", "fairscale", "opencv-python", "matplotlib", "pycocotools", "pywavelets"
]
subprocess.run([sys.executable, "-m", "pip", "install"] + pypi_packages, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. FLASH ATTENTION (Lourd, check avant)
if is_installed("flash_attn"):
    print("⚡ Flash Attention : DÉJÀ INSTALLÉ.")
else:
    print("⚡ Compilation Flash Attention... (Patience)")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "flash-attn", "--no-build-isolation"])
    except Exception as e:
        print(f"⚠️ Skip Flash-Attn (Erreur non critique): {e}")

# 6. LLAMA-CPP (Lourd, check avant)
if is_installed("llama_cpp"):
    print("🦙 Llama-CPP : DÉJÀ INSTALLÉ.")
else:
    print("🦙 Compilation Llama-CPP... (Patience)")
    try:
        env = os.environ.copy()
        env["CMAKE_ARGS"] = "-DGGML_CUDA=on"
        subprocess.run([sys.executable, "-m", "pip", "install", "llama-cpp-python", "--no-cache-dir"], env=env, check=True)
    except Exception: pass

# 7. DEPENDANCES GIT & SAM2
print("🐙 Vérification dépendances Git...")
git_packages = [
    "git+https://github.com/WASasquatch/cstr.git",
    "git+https://github.com/WASasquatch/img2texture.git",
    "git+https://github.com/WASasquatch/ffmpy.git",
    "git+https://github.com/facebookresearch/sam2.git"
]
# On lance l'install sans forcer, pip gérera si c'est déjà là
subprocess.run([sys.executable, "-m", "pip", "install"] + git_packages + ["--no-build-isolation"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 8. MODÈLES INSIGHTFACE
print("📥 Vérification InsightFace (AntelopeV2)...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]
missing_files = [f for f in files_antelope if not os.path.exists(f"{antelope_dir}/{f}")]

if missing_files:
    print(f"   -> Téléchargement de {len(missing_files)} modèles manquants...")
    for f in missing_files:
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
else:
    print("   -> Modèles déjà présents.")

# 9. LANCEMENT & DÉTECTION
print("\n" + "="*60)
runpod_id = os.environ.get("RUNPOD_POD_ID")
vast_ip = os.environ.get("PUBLIC_IPADDR")
if runpod_id:
    print(f"🔵 DÉTECTION : RUNPOD")
    print(f"🔥 TON LIEN : https://{runpod_id}-8188.proxy.runpod.net")
elif vast_ip:
    print(f"🟢 DÉTECTION : VAST.AI")
    print(f"🔥 TON LIEN : http://{vast_ip}:8188")
else:
    print("⚪ DÉTECTION : LOCAL")
    print("👉 Ouvre : http://localhost:8188")
print("="*60 + "\n")

print("🚀 Lancement du serveur...")
run_script = """#!/bin/bash
cd /workspace/ComfyUI
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet --enable-cors-header
"""
with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

!./run_comfy.sh

In [ ]:
INSTALLATION VENV & DEPENDANCES + LANCEMENT COMFYUI

In [ ]:
# 🚀 DÉMARRAGE V21.4 - ULTIMATE COMMANDER (TOTAL FIX: OLLAMA + CV2 + CREEPY + IM7)
import os, subprocess, sys, shutil, torch, threading, time, re, json

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
VENV_DIR = "/workspace/my_env"
PYTHON_EXEC = f"{VENV_DIR}/bin/python"
PIP_EXEC = f"{VENV_DIR}/bin/pip"

os.environ["PIP_ROOT_USER_ACTION"] = "ignore"
os.environ["PYTHONNOUSERSITE"] = "1"
os.environ["DEBIAN_FRONTEND"] = "noninteractive"
os.environ["MPLBACKEND"] = "Agg"

def is_installed_in_venv(package_name):
    try:
        subprocess.check_call([PYTHON_EXEC, "-m", "pip", "show", package_name], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except: return False

print(f"💎 DÉMARRAGE V21.4 - ULTIMATE COMMANDER")

# ==================================================================================
# 1. SYSTÈME & NETTOYAGE
# ==================================================================================
print("\n🔧 [1/5] Nettoyage et préparation système...")
subprocess.run(f"find {COMFY_DIR}/custom_nodes -name '.ipynb_checkpoints' -type d -exec rm -rf {{}} +", shell=True)
subprocess.run("cp /etc/apt/trusted.gpg /usr/share/keyrings/nvidia-keyring.gpg || true", shell=True, stderr=subprocess.DEVNULL)

sys_pkgs = [
    "ncdu", "ffmpeg", "git", "cmake", "aria2", "libsndfile1", "libsndfile1-dev", 
    "build-essential", "pkg-config", "libpng-dev", "libjpeg-dev", "libtiff-dev", "libwebp-dev",
    "libtool", "libltdl-dev", "python3-venv", "python3-pip", "libharfbuzz-dev", "libfribidi-dev"
]
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y"] + sys_pkgs, stdout=subprocess.DEVNULL)

# ImageMagick 7 (Compilation si absent)
if not os.path.exists("/usr/local/bin/magick"):
    print("    -> 🛠️ Compilation ImageMagick 7...")
    os.chdir("/tmp")
    subprocess.run("rm -rf ImageMagick-7", shell=True)
    subprocess.run(["git", "clone", "--depth", "1", "https://github.com/ImageMagick/ImageMagick.git", "ImageMagick-7"], stdout=subprocess.DEVNULL)
    os.chdir("ImageMagick-7")
    subprocess.run("./configure --with-modules --with-quantum-depth=16 --enable-hdri --disable-docs", shell=True, stdout=subprocess.DEVNULL)
    subprocess.run(f"make -j{os.cpu_count()}", shell=True, stdout=subprocess.DEVNULL)
    subprocess.run(["make", "install"], stdout=subprocess.DEVNULL)
    subprocess.run(["ldconfig", "/usr/local/lib"], stdout=subprocess.DEVNULL)
    os.chdir("/workspace")

# ==================================================================================
# 2. PATCHES CREEPYNODES & GEMINI CONFIG
# ==================================================================================
creepy_init = f"{COMFY_DIR}/custom_nodes/comfyui_creepy_nodes/__init__.py"
if os.path.exists(creepy_init):
    with open(creepy_init, 'r') as f:
        content = f.read()
    if "Creepy_directors" in content:
        new_content = content.replace("Creepy_directors", "creepy_directors")
        with open(creepy_init, 'w') as f:
            f.write(new_content)
        print("    -> ✅ Patch CreepyNodes Global appliqué.")

creepy_json = f"{COMFY_DIR}/custom_nodes/comfyui_creepy_nodes/assets/scripts/api_keys_config.json"
gemini_txt = f"{COMFY_DIR}/custom_nodes/comfyui_creepy_nodes/assets/scripts/gemini_api_key.txt"
if os.path.exists(os.path.dirname(creepy_json)):
    config_data = {"Gemini Free": gemini_txt, "Gemini Paid": gemini_txt}
    with open(creepy_json, 'w') as f:
        json.dump(config_data, f, indent=4)
    print("    -> ✅ Configuration JSON Gemini mise à jour.")

# ==================================================================================
# 3. VENV & INSTALLATIONS
# ==================================================================================
if not os.path.exists(VENV_DIR):
    print(f"\n🌱 [2/5] Création du VENV...")
    subprocess.check_call([sys.executable, "-m", "venv", VENV_DIR])

print("\n📦 [3/5] Installation des librairies...")
def run_pip_visible(args_list, no_isolation=False):
    base_cmd = [PYTHON_EXEC, "-m", "pip", "install", "--no-cache-dir", "--prefer-binary"]
    if no_isolation: base_cmd.append("--no-build-isolation")
    subprocess.check_call(base_cmd + args_list)

subprocess.run([PYTHON_EXEC, "-m", "ensurepip", "--upgrade"], stdout=subprocess.DEVNULL)
run_pip_visible(["--upgrade", "pip", "wheel", "setuptools", "ninja", "packaging"])

# FIX OPENCV (Nettoyage agressif pour LayerStyle)
print("    -> Fix OpenCV (Nettoyage et réinstallation Contrib)...")
subprocess.run([PYTHON_EXEC, "-m", "pip", "uninstall", "-y", "opencv-python", "opencv-python-headless", "opencv-contrib-python", "opencv-contrib-python-headless"], stdout=subprocess.DEVNULL)
run_pip_visible(["opencv-contrib-python"])

print("    -> Installation requirements officiels...")
run_pip_visible(["-r", f"{COMFY_DIR}/requirements.txt"])

pypi_packages = [
    "transformers>=4.48.0", "accelerate", "webcolors", "imageio-ffmpeg", "matplotlib",
    "rembg", "onnxruntime-gpu", "Wand", "insightface==0.7.3", "kornia", "spandrel",
    "color-matcher", "gguf", "segment-anything", "diffusers", "ultralytics",
    "rotary_embedding_torch", "dill", "timm", "simpleeval", "blend_modes",
    "facenet-pytorch", "mediapipe", "fvcore", "yapf", "omegaconf",
    "soundfile", "librosa", "einops", "duckdb", "piexif", "pywavelets",
    "google-generativeai", "google-genai", "scipy", "lark", "facexlib",
    "ftfy", "xformers", "openai"
]
run_pip_visible(pypi_packages)

# SageAttention & SAM2
if not is_installed_in_venv("sageattention"):
    print("    -> 🚀 SageAttention 2...")
    try: run_pip_visible(["git+https://github.com/thu-ml/SageAttention.git"])
    except: run_pip_visible(["sageattention"])

if not is_installed_in_venv("sam2"):
    print("    -> 🎭 SAM2...")
    try: run_pip_visible(["git+https://github.com/facebookresearch/sam2.git"], no_isolation=True)
    except: pass

# ==================================================================================
# 4. MODÈLES FACE (ANTELOPE V2)
# ==================================================================================
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
if not os.path.exists(antelope_dir):
    os.makedirs(antelope_dir, exist_ok=True)
    print("\n📥 [4/5] Téléchargement modèles Face...")
    for f in ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]:
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL)

# ==================================================================================
# 5. LANCEMENT
# ==================================================================================
print("\n🚀 [5/5] Lancement ComfyUI...")

runpod_id = os.environ.get("RUNPOD_POD_ID")
if runpod_id: print(f"\n🔗 LIEN RUNPOD : https://{runpod_id}-8188.proxy.runpod.net")

run_script = f"""#!/bin/bash
export PATH="{VENV_DIR}/bin:/usr/local/bin:$PATH"
export MAGICK_HOME="/usr/local"
export LD_LIBRARY_PATH="/usr/local/lib:$LD_LIBRARY_PATH"
export PYTHONNOUSERSITE=1
export MPLBACKEND=Agg
cd {COMFY_DIR}
python main.py --listen 0.0.0.0 --port 8188 --bf16-unet --enable-cors-header --preview-method auto
"""
with open("run_comfy.sh", "w") as f: f.write(run_script)
os.chmod("run_comfy.sh", 0o755)
!./run_comfy.sh

💎 DÉMARRAGE V21.4 - ULTIMATE COMMANDER

🔧 [1/5] Nettoyage et préparation système...
    -> ✅ Configuration JSON Gemini mise à jour.

📦 [3/5] Installation des librairies...
    -> Fix OpenCV (Nettoyage et réinstallation Contrib)...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 MB 170.6 MB/s  0:00:00 eta 0:00:01
    -> Installation requirements officiels...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 151.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 129.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [openai]2m3/4 [openai]python]
    -> 🎭 SAM2...
  Cloning https://github.com/facebookresearch/sam2.git to /tmp/pip-req-build-kw_xvh87


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam2.git /tmp/pip-req-build-kw_xvh87


  Resolved https://github.com/facebookresearch/sam2.git to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'

🚀 [5/5] Lancement ComfyUI...

🔗 LIEN RUNPOD : https://cjlpqedbvtkemj-8188.proxy.runpod.net
[SAM3] ComfyUI-SAM3 prestartup script running...
[SAM3] Script directory: /workspace/ComfyUI/custom_nodes/comfyui-sam3
[SAM3] ComfyUI root: /workspace/ComfyUI
[SAM3] Copying image assets to /workspace/ComfyUI/input...
[SAM3] Image assets: 0 copied, 4 skipped
[SAM3] Prestartup script completed
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-14 13:56:37.267
** Platform: Linux
** Python version: 3.11.10 (main, Sep  7 2024, 18:35:41) [GCC 11.4.0]
** Python executable: /workspace/my_env/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspa